# Convergence

In this activity, we explore the notion of convergence for iterative algorithms.


In [ ]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import hadamard
from scipy.stats import unitary_group, ortho_group
import pandas as pd

## Underdetermined Linear Systems

Consider the following underdetermined system of linear equations, where (wide) matrix $A: \mathbb{R}^m \rightarrow \mathbb{R}^n$ is a linear transformation and
$$\mathbf{y} = A \mathbf{x} .$$
Also, we are given the information that $\mathbf{x}$ is a sparse vector, with only $K = 8$ non-zero entries.
These non-zero entries are all ones.

In [ ]:
np.random.seed(2021)

dim = 128
K = 8
samples = int(np.ceil(K * np.log(dim)))
samples = 56

In [ ]:
matrix_df = pd.read_csv('matrix.csv')
A = matrix_df.values
observation_df = pd.read_csv('observation.csv')
y = observation_df.values

## Composite Iterative Algorithm

The algorithm is initialized with $\mathbf{x}^{(0)}$ being the all-zero vector.
The algorithm then proceeds by alternating between the following two steps:
$$\mathbf{z}^{(t)} = \mathbf{y} - A \mathbf{x}^{(t)}$$
$$\mathbf{x}^{(t+1)} = \gamma A^{\mathrm{T}} \mathbf{z}^{(t)} + \mathbf{x}^{(t)}$$

In [ ]:
def CompositeStep(A_loc,y_loc,xt_loc,gamma):
    zt_loc = y_loc - np.matmul(A_loc,xt)
    return (gamma*np.matmul(A_loc.transpose(),zt_loc) + xt_loc)

gamma = 0.25
xt = np.zeros(shape=(A.shape[1],1))
for iteration in range(200):
    xt = CompositeStep(A,y,xt,gamma)
print(np.linalg.norm(np.matmul(A,xt)-y))

### Questions
 1. What is the interval of parameter `gamma` $\gamma > 0$ for which this algorithm converges?
     * **Answer:**
 2. Can you infer the sparse pattern associated with the true `x`?
     * **Answer:** Yes or No
 3. Would the interval you have identified above hold for any `y`, or just for the one selected for this challenge?
     * **Answer:**

## Iterative Soft Tresholding

We consider this alternative.
The algorithm is initialized with $\mathbf{x}^{(0)}$ being the all-zero vector.
The algorithm then proceeds by alternating between the following two steps:
$$\mathbf{z}^{(t)} = \mathbf{y} - A \mathbf{x}^{(t)}$$
$$\mathbf{x}^{(t+1)} = \eta \left( A^{\mathrm{T}} \mathbf{z}^{(t)} + \mathbf{x}^{(t)} \right)$$
where $\eta(\cdot)$ is a soft threshold.

In [ ]:
def SoftThreshold(r,threshold):
    r_abs = np.absolute(r)
    r_sign = np.sign(r)
    return (r_sign * np.maximum(np.zeros(r.shape), r_abs - threshold*np.ones(r.shape)))

def CompositeStepST(A,y,xt,threshold):
    zt = y - np.matmul(A,xt)
    rt = np.matmul(np.transpose(A),zt) + xt
    xt = SoftThreshold(rt,threshold)
    return xt

In [ ]:
threshold = 0.01

xt = np.zeros(shape=(A.shape[1],1))
for iteration in range(10000):
    xt = CompositeStepST(A,y,xt,threshold)
print(np.linalg.norm(np.matmul(A,xt)-y))

### Questions
 1. Can you infer the sparse pattern associated with the true `x`?
     * **Answer:** Yes or No
 2. If this is the case, why would that be?
     * **Answer:**